# Introduction

This chapter will be concerned with joining the `ingredients_df` of the `recipe_db` to the appropriate `food_df` entry of the `molecule_db`.

# Setup

In [1]:
#|default_exp molecule.match

In [2]:
#| export
from pyprojroot import here
root = here()
import sys
sys.path.append(str(root))

In [3]:
#|export
import pandas as pd

from recipe_dataset.density.food_match import *
from recipe_dataset.utils.utils import *
from recipe_dataset.utils.join_utils import *

import json
from tqdm import tqdm

In [4]:
#| export
with open(f'{root}/config/default_words.json', 'r') as f:
    default_words = json.load(f)['molecule']

with open(f'{root}/config/exclusion_words.json', 'r') as f:
    exclusion_words = json.load(f)['molecule']

In [5]:
ingredients_df = pd.read_feather('../../data/local/recipe/partial/ingredients/0.feather')
expanded_ingredients_df = pd.read_feather('../../data/local/recipe/partial/expanded_ingredients/0.feather')

In [6]:
food_df = pd.read_feather('../../data/local/molecule/full/food/2_feature_engineered.feather')
content_df = pd.read_feather('../../data/local/molecule/full/content/1_processed.feather')
compound_df = pd.read_feather('../../data/local/molecule/full/compound/1_processed.feather')

# Joining

## Overview

Following the same process as the recipes_db:density_db joining. 

This case is quite a lot more simple as the food options are already parsed and sussinct. It's much easier to hone down a search, with much less pitfalls to keep in mind.

## Default Override

To aid with the search here, we need to introduce a mechanism in the search to either point the search to the right place, or transform the ingredient into something that the search will ultimately find. 

There are two design decisions to decide on:
1. ingredient seleciton (which ingredient gets override)
2. type of output (direct food_id match or transformed ingredient)

### Ingredient Selection

To decide on this, let's start with the criteria that we want to match:

- ingredient root synonyms (breads, pastas, chicken parts, liquor)
- ingredients that don't follow the regular rules
    - priority noun at at the end (egg whites, lime juice)

Looking at these, these are two different problems. The first requires a synonym check, which might be best to reserve for doing after the first join, if none are found. The latter is more what we are looking at here. We make our rules following general language patterns, however there will be irregularities, where we need this override to patch together.

How should this be done? We need to define a criteria for a match, we could begin generally, and work our way to being more specific as it begins to grab too much.

### Output Type

It would be the most efficient to get the food_id directly. However there is the issue that this would be more difficult to manually enter as the database would need to be known. It might be possible that the efficiency that it saves (marginal compute time on the full dataframe join) wouldn't compensate for the extra difficulty to enter in.

Lets just transform the ingredient, as this is better for enterring in, both for ease and clarity.

In [7]:
#| export
with open(f'{root}/config/molecule/ingredient_transforms.json', 'r') as f:
    default_transforms = json.load(f)

In [8]:
#| export
def default_transform(ingredient):
    for defualt_instance in default_transforms.values():
        if all([key_word in ingredient.values for key_word in defualt_instance['key']]):
            return pd.Series(defualt_instance['value'], dtype='string', name=ingredient.name)
    return ingredient

In [9]:
# converting lime juice into lime
ingredient = expanded_ingredients_df.loc[894410, 7]
ingredient = ingredient[ingredient.notnull()]
ingredient

name.name.nouns.1    juice
name.name.nouns.0     lime
Name: (894410, 7), dtype: object

In [10]:
assert default_transform(ingredient).values == ['lime']

## Exlcuding Words

Some words cause more harm than good, and they do differ from the datasets.

Example here is 'dry' as this dataset doesn't consist of separate ingredients for dried/not dried. Rather it has an ingredient, and contains sub-foods in this, with some exception which is what causes isse. We don't want 'dried mango' to be matching with 'dried milk'.

We have previously filtered this out in the selection process, but it makes more sense to simply do this before the first search here.

In [11]:
ingredient = expanded_ingredients_df.loc[1719483, 14]
ingredient = ingredient[ingredient.notnull()]
ingredient

name.name.nouns.0            prune
name.description.others.0      dry
Name: (1719483, 14), dtype: object

In [12]:
ingredient[~ingredient.isin(exclusion_words)]

name.name.nouns.0    prune
Name: (1719483, 14), dtype: object

## Food DF Search

In [13]:
#| export 
def fuzzy_search_words(s, search_word):
    for w in s.split(' '):
        if fuzzy_search(w, search_word, 85):
            return True
    return False

The threshold for might be too low. We are seeing matches when there shouldn't be.

In [14]:
fuzzy_search('flake', 'lake', 85)

True

In [15]:
#| export
def find_ingredient_food_df_matches(ingredient, food_names, debug=False):

    ingredient = ingredient[ingredient.notnull()]
    ingredient = ingredient[~ingredient.isin(exclusion_words)]
    ingredient = default_transform(ingredient)
    
    matched_food_df = food_names.copy(deep=True)
    matched_idxs = matched_food_df.index
    found_match = False

    if debug: debug_idxs = {col: {} for col in ingredient.index}

    for search_col, search_word in ingredient.items():
        
        current_matched_idxs = matched_food_df.index[matched_food_df.apply(fuzzy_search_words, args=(search_word,))]

        if debug: debug_idxs[search_col] = {'value': search_word, 'size': len(current_matched_idxs), 'idxs': {'matched': current_matched_idxs, 'selected': []}}

        if not current_matched_idxs.empty:
            matched_idxs = current_matched_idxs
            found_match = True
        else:
            if not search_col.startswith('name.name.nouns'):
                pass

        matched_food_df = matched_food_df.loc[matched_idxs]

        if debug: debug_idxs[search_col]['idxs']['selected'] = matched_idxs

    if not found_match: matched_idxs = []
    
    if debug:
        return matched_idxs, debug_idxs
    else:
        return matched_idxs

In [16]:
ingredient = expanded_ingredients_df.loc[1746116, 0]
ingredient[ingredient.notnull()]

name.name.nouns.0           butter
name.description.nouns.2      lake
name.description.nouns.0      land
Name: (1746116, 0), dtype: object

In [17]:
matched_idxs, debug_idxs = find_ingredient_food_df_matches(ingredient, food_df['name'], debug=True)
matched_df = food_df.loc[matched_idxs]
matched_df

,base_food_id,name,name_scientific,description,wikipedia_id,food_group,food_subgroup,food_type,category,n_content
id,,,,,,,,,,
805,667,butter,<NA>,Butter is a dairy product made by churning fre...,Butter,Milk and milk products,Other milk products,Type 2,specific,703
806,668,butter substitute,<NA>,None,<NA>,Baking goods,Substitutes,Type 2,specific,53
850,706,cocoa butter,<NA>,"Cocoa butter, also called theobroma oil, is a ...",Cocoa_butter,Cocoa and cocoa products,Cocoa products,Type 2,specific,244


## Selecting Matches

In [18]:
#| export
def calculate_match_stats(match_string, ingredient_values):

    match_words = match_string.split(' ')

    match_position = 99
    match_count = 0
    whole_match_count = 0
    word_count = 99
    default_word_count = 0
    exclusion_word_count = 0

    for i, match_word in enumerate(match_words):
        if any([fuzzy_search(match_word, ingredient_value) for ingredient_value in ingredient_values]):
            match_position = i
        match_count += len([ingredient_value for ingredient_value in ingredient_values if fuzzy_search(match_word, ingredient_value)])
        whole_match_count += len([ingredient_value for ingredient_value in ingredient_values if contains_whole_word(match_word, ingredient_value)])
        if match_word in default_words: default_word_count += 1
        if match_word in exclusion_words: exclusion_word_count += 1

    word_count = len(match_words) - default_word_count
    
    return (
        whole_match_count,
        match_count,
        match_position,
        default_word_count,
        word_count,
    )

In [19]:
#| export
def select_from_matches(ingredient, matched_df_names, return_df=False):

    # since we don't have any rubbish foods here, we can include all search terms of the ingredient
    ingredient_cols = ingredient.index[ingredient.notnull()]
    ingredient_values = ingredient[ingredient_cols].values

    matched_df = matched_df_names.to_frame('name')

    matched_df['whole_match_count'], \
    matched_df['match_count'], \
    matched_df['match_position'], \
    matched_df['default_word_count'], \
    matched_df['word_count'] = zip(*matched_df['name'].apply(calculate_match_stats, args=(ingredient_values,)))

    matched_df = matched_df.sort_values(
        ['whole_match_count',
         'match_count',
         'match_position',
         'word_count',
         'default_word_count',
         'id'],
        ascending = [
            False,
            False,
            True,
            True,
            False,
            True
        ]
    )

    if return_df:
        return matched_df
    else:
        return matched_df.iloc[0].name if not matched_df.empty else pd.NA

In [20]:
select_from_matches(ingredient, matched_df['name'], True)

,name,whole_match_count,match_count,match_position,default_word_count,word_count
id,,,,,,
805,butter,1,1,0,0,1
806,butter substitute,1,1,0,0,2
850,cocoa butter,1,1,1,0,2


In [21]:
#| export
def match_ingredient(ingredient, food_names):
    matched_names = food_names.loc[find_ingredient_food_df_matches(ingredient, food_names)]
    if matched_names.empty: return pd.NA
    selected_idx = select_from_matches(ingredient, matched_names)
    return selected_idx

In [22]:
food_ids = expanded_ingredients_df.progress_apply(match_ingredient, axis=1, args=(food_df['name'],))
food_ids = food_ids.rename('food_id')

  0%|                                                                                                                                                                                                                                                            | 0/2450 [00:00<?, ?it/s]

  1%|██▊                                                                                                                                                                                                                                               | 28/2450 [00:00<00:08, 277.11it/s]

  2%|█████▌                                                                                                                                                                                                                                            | 56/2450 [00:00<00:09, 259.16it/s]

  3%|████████                                                                                                                                                                                                                                          | 82/2450 [00:00<00:09, 257.61it/s]

  4%|██████████▌                                                                                                                                                                                                                                      | 108/2450 [00:00<00:09, 256.08it/s]

...

 96%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉         | 2358/2450 [00:09<00:00, 252.51it/s]

 97%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌      | 2384/2450 [00:09<00:00, 254.08it/s]

 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████    | 2410/2450 [00:09<00:00, 251.79it/s]

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 2436/2450 [00:09<00:00, 252.76it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2450/2450 [00:09<00:00, 255.28it/s]

In [23]:
results_df = ingredients_df.join(food_ids).join(food_df['name'], on='food_id').drop(['quantity', 'unit', 'ingredient_string', 'unit_tags', 'unit_remainders'], axis=1)
results_df.head() 

name.name   name.description       comment preparation  \
recipe  ingredient                                                          
1746116 0             butter   land lake butter          <NA>    softened   
        1              sugar              sugar          <NA>        <NA>   
        2                egg      land lake egg  (yolks only)        <NA>   
        3            vanilla            vanilla          <NA>        <NA>   
        4              flour  all-purpose flour          <NA>        <NA>   

                   unit_type food_id     name  
recipe  ingredient                             
1746116 0             volume     805   butter  
        1             volume     809    sugar  
        2            portion     773      egg  
        3             volume     269  vanilla  
        4             volume     971    flour

In [24]:
pd.set_option('display.max_rows', None)

In [25]:
results_df.head(200)

name.name  \
recipe  ingredient                                  
1746116 0                                  butter   
        1                                   sugar   
        2                                     egg   
        3                                 vanilla   
        4                                   flour   
        5                                 caramel   
        6                                   cream   
        7                                   pecan   
        8               semi-sweet chocolate chip   
        9                              shortening   
1828339 0                              grape leaf   
        2                         ground turmeric   
        3                   long grain brown rice   
        4                           vegetable oil   
        5                                   onion   
        6                           dried apricot   
        7                                 sultana   
        8                         ground cinnamon   
        9                         ground allspice   
        10                                   mint   
        11                            lemon juice   
        12                           orange juice   
        13                                  water   
1703    0                          pkg applesauce   
        1                                     egg   
        2                                   water   
        3                               mincemeat   
        4                                     nut   
1608197 0                                  orange   
        1                                   lemon   
        2                                   water   
        3                                   sugar   
382666  0                          chicken breast   
        1                                  butter   
        2                           parsley flake   
        3                       herb stuffing mix   
        4                         parmesan cheese   
894410  0                                   water   
        1                              white rice   
        2                              black bean   
        3                                  tomato   
        4                           green chilies   
        5                          barbecue sauce   
        6                                   cumin   
        7                              lime juice   
        8                      frozen corn kernel   
        9                                    lime   
231060  0                                  butter   
        1                                   sugar   
        2                                   cocoa   
        3                                  coffee   
        4                                 vanilla   
        5                              rolled oat   
1262123 0                                linguine   
        1                             heavy cream   
        2           store-bought red pepper pesto   
        3                                   chive   
        4                                  salmon   
317211  0                                   flour   
        1                                   pecan   
        2                                  butter   
        3                            cream cheese   
        4                          powdered sugar   
        5                     container cool whip   
        6               pistachio instant pudding   
        7                                    milk   
        8                                   pecan   
824709  0                                   bread   
        1                         ground cinnamon   
        2                                   sugar   
        3                               margarine   
855127  0                                eggplant   
        1                                  tomato   
        2                         parmesan cheese   
        3   

In [26]:
pd.reset_option('display.max_rows')

# NA Values

In [27]:
na_results_df = results_df[results_df['food_id'].isna()]
na_results_df.shape[0] / results_df.shape[0]

0.062448979591836734

In [28]:
na_expanded_ingredients_df = expanded_ingredients_df.loc[na_results_df.index]
na_expanded_ingredients_df

name.name.nouns.4 name.name.nouns.3 name.name.nouns.2  \
recipe  ingredient                                                         
1746116 5                       None              None              None   
1828339 7                       None              None              None   
1703    0                       None              None              None   
        3                       None              None              None   
1262123 0                       None              None              None   
...                              ...               ...               ...   
1097688 0                       None              None              None   
        8                       None              None              None   
1748614 2                       None              None              None   
1936429 3                       None              None              None   
1401651 4                       None              None              None   

                   name.name.nouns.1 name.name.nouns.0 name.name.others.0  \
recipe  ingredient                                                          
1746116 5                       None           caramel               None   
1828339 7                       None           sultana               None   
1703    0                 applesauce               pkg               None   
        3                       None         mincemeat               None   
1262123 0                       None          linguine               None   
...                              ...               ...                ...   
1097688 0                       None            batter               None   
        8                       None              soda             baking   
1748614 2                    glucose            liquid               None   
1936429 3                       None             fresh               None   
1401651 4                     hanout                ra                 el   

                   name.name.others.1 name.name.others.2 name.name.others.3  \
recipe  ingredient                                                            
1746116 5                        None               None               None   
1828339 7                        None               None               None   
1703    0                        None               None               None   
        3                        None               None               None   
1262123 0                        None               None               None   
...                               ...                ...                ...   
1097688 0                        None               None               None   
        8                        None               None               None   
1748614 2                        None               None               None   
1936429 3                        None               None               None   
1401651 4                        None               None               None   

                   name.description.nouns.5  ... name.description.nouns.3  \
recipe  ingredient                           ...                            
1746116 5                              None  ...                     None   
1828339 7                              None  ...                     None   
1703    0                              None  ...                     None   
        3                              None  ...                     None   
1262123 0                              None  ...                     None   
...                                     ...  ...                      ...   
1097688 0                              None  ...                     None   
        8                              None  ...                     None   
1748614 2                              None  ...                     None   
1936429 3                              None  ...                     None   
1401651 4                              None  ...                     None   

In [29]:
#| export
def create_na_synonyms_df(na_expanded_ingredients_df):

    na_synonyms_list_df = na_expanded_ingredients_df.copy(deep=True)
    na_synonyms_df = pd.DataFrame(dtype='string')

    # fetching thesaurus synonyms
    for col in tqdm(na_synonyms_list_df.columns):
        na_synonyms_list_df[col] = na_synonyms_list_df[col].apply(find_alt_words)
        na_synonyms_list_df[col] = na_synonyms_list_df[col].apply(lambda x: [] if not isinstance(x, list) else x)

    # expanding synonym lists into columns
    for col in na_synonyms_list_df.columns:
        expanded = pd.DataFrame(na_synonyms_list_df[col].tolist(), index=na_synonyms_list_df.index, dtype='string').loc[:,:8]
        expanded.columns = [col + '.' + str(c) for c in expanded.columns]
        na_synonyms_df = pd.concat([na_synonyms_df, expanded], axis=1)

    return na_synonyms_df

In [30]:
na_synonyms_df = create_na_synonyms_df(na_expanded_ingredients_df)
na_synonyms_df

  0%|                                                                                                                                                                                                                                                              | 0/21 [00:00<?, ?it/s]

 14%|███████████████████████████████████▏                                                                                                                                                                                                                  | 3/21 [00:00<00:03,  5.75it/s]

 33%|██████████████████████████████████████████████████████████████████████████████████                                                                                                                                                                    | 7/21 [00:00<00:01, 13.11it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 33.21it/s]

name.name.nouns.1.0 name.name.nouns.1.1  \
recipe  ingredient                                           
1746116 5                         <NA>                <NA>   
1828339 7                         <NA>                <NA>   
1703    0                        sauce               apple   
        3                         <NA>                <NA>   
1262123 0                         <NA>                <NA>   
...                                ...                 ...   
1097688 0                         <NA>                <NA>   
        8                         <NA>                <NA>   
1748614 2                   aldohexose              aldose   
1936429 3                         <NA>                <NA>   
1401651 4                         <NA>                <NA>   

                   name.name.nouns.1.2 name.name.nouns.1.3  \
recipe  ingredient                                           
1746116 5                         <NA>                <NA>   
1828339 7                         <NA>                <NA>   
1703    0                         dish           nutriment   
        3                         <NA>                <NA>   
1262123 0                         <NA>                <NA>   
...                                ...                 ...   
1097688 0                         <NA>                <NA>   
        8                         <NA>                <NA>   
1748614 2                       hexose      monosaccharide   
1936429 3                         <NA>                <NA>   
1401651 4                         <NA>                <NA>   

                   name.name.nouns.1.4 name.name.nouns.0.0  \
recipe  ingredient                                           
1746116 5                         <NA>               candy   
1828339 7                         <NA>               grape   
1703    0                         <NA>                <NA>   
        3                         <NA>          concoction   
1262123 0                         <NA>            linguini   
...                                ...                 ...   
1097688 0                         <NA>          concoction   
        8                         <NA>               water   
1748614 2               monosaccharide               fluid   
1936429 3                         <NA>                <NA>   
1401651 4                         <NA>              radium   

                   name.name.nouns.0.1 name.name.nouns.0.2  \
recipe  ingredient                                           
1746116 5                        sweet              dainty   
1828339 7                     vinifera               grape   
1703    0                         <NA>                <NA>   
        3                    foodstuff                <NA>   
1262123 0                        pasta                <NA>   
...                                ...                 ...   
1097688 0                    foodstuff                <NA>   
        8                         soda               tonic   
1748614 2                    substance              matter   
1936429 3                         <NA>                <NA>   
1401651 4                           Ra                  88   

                   name.name.nouns.0.3 name.name.nouns.0.4  ...  \
recipe  ingredient                                          ...   
1746116 5                    nutriment                <NA>  ...   
1828339 7                        fruit              edible  ...   
1703    0                         <NA>                <NA>  ...   
        3                         <NA>                <NA>  ...   
1262123 0                         <NA>                <NA>  ...   
...                                ...                 ...  ...   
1097688 0                         <NA>                <NA>  ...   
        8                          pop                 pop  ...   
1748614 2                         part              entity  ...   
1936429 3                         <NA>                

In [31]:
assert len(na_synonyms_df) == len(na_expanded_ingredients_df)

In [32]:
orig = na_expanded_ingredients_df.iloc[0]
synonym = na_synonyms_df.iloc[0]
orig[orig.notnull()], synonym[synonym.notnull()]

(name.name.nouns.0    caramel
 Name: (1746116, 5), dtype: object,
 name.name.nouns.0.0        candy
 name.name.nouns.0.1        sweet
 name.name.nouns.0.2       dainty
 name.name.nouns.0.3    nutriment
 Name: (1746116, 5), dtype: string)

In [33]:
na_synonyms_df['food_id'] = na_synonyms_df.progress_apply(match_ingredient, axis=1, args=(food_df['name'],))
na_synonyms_df['food_id'].isna().sum(), results_df['food_id'].isna().sum()

  0%|                                                                                                                                                                                                                                                             | 0/153 [00:00<?, ?it/s]

 12%|████████████████████████████▌                                                                                                                                                                                                                      | 18/153 [00:00<00:00, 177.10it/s]

 24%|█████████████████████████████████████████████████████████▏                                                                                                                                                                                         | 36/153 [00:00<00:00, 169.96it/s]

 35%|█████████████████████████████████████████████████████████████████████████████████████▊                                                                                                                                                             | 54/153 [00:00<00:00, 141.14it/s]

 49%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                                            | 75/153 [00:00<00:00, 161.95it/s]

...

 76%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                          | 116/153 [00:00<00:00, 176.60it/s]

 88%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                            | 135/153 [00:00<00:00, 161.98it/s]

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 152/153 [00:00<00:00, 152.52it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 153/153 [00:00<00:00, 157.66it/s]

(32, 153)

Nice, this is finding the majority of the missing ingredients.

In [34]:
results_df['food_id'].fillna(na_synonyms_df['food_id'], inplace=True)
results_df['food_id'].isna().sum()

32

In [35]:
results_df.drop('name', inplace=True, axis=1)
results_df = results_df.join(food_df['name'], on='food_id')

## Evaluation

In [36]:
pd.set_option('display.max_rows', None)

In [37]:
results_df.loc[na_results_df.index]

name.name                 name.description  \
recipe  ingredient                                                           
1746116 5                         caramel                          caramel   
1828339 7                         sultana                          sultana   
1703    0                  pkg applesauce                   pkg applesauce   
        3                       mincemeat                        mincemeat   
1262123 0                        linguine                         linguine   
1428429 0                         tequila                          tequila   
767911  9                         pimento                          pimento   
53081   4                     baking soda                      baking soda   
2196831 4              limoncello liqueur               limoncello liqueur   
        6                        baguette                         baguette   
1653851 3                creole seasoning                 creole seasoning   
689193  4                      mozzarella                       mozzarella   
811610  2                      mayonnaise                       mayonnaise   
939793  2                     vinaigrette                      vinaigrette   
        6                           bacon                   crumbled bacon   
1956519 4                       pepperoni                        pepperoni   
519070  1                            soda                             soda   
115650  2                        pimiento                         pimiento   
1805929 6                        parmesan                         parmesan   
222     1                     baking soda                      baking soda   
661227  0                      mayonnaise                       mayonnaise   
        4                      beau monde                       beau monde   
1546037 17                  half-and-half                    half-and-half   
820354  0                            oleo                             oleo   
606258  5                          catsup                           catsup   
745912  0                         doritos                      pkg doritos   
806445  5                     baking soda                      baking soda   
869338  4                           tater                        tater tot   
1442064 3           culinary lavender bud      dried culinary lavender bud   
1345852 5                      mayonnaise                       mayonnaise   
279384  7              prepared mincemeat               prepared mincemeat   
1450961 6                fajita seasoning                 fajita seasoning   
1916172 5                        linguine                         linguine   
1169604 0                       gremolata                        gremolata   
1043275 1                         pimento                          pimento   
23612   1                          catsup                           catsup   
683481  3                          jell-o                       pkg jell-o   
1944911 2                ornage marmalade                 ornage marmalade   
1396331 0                  guajillo chile                   guajillo chile   
        2                        rutabaga                         rutabaga   
520540  0                     cornish hen                      cornish hen   
1806301 1               poultry seasoning                poultry seasoning   
1257944 0                       none none                        none none   
        9                       none none                        none none   
55647   1                        macaroni                         macaroni   
1883505 4                         nagaimo                          nagaimo   
        6                      mayonnaise                       mayonnaise   
1419202 1                commercial pesto                 commercial pesto   
1369744 0                        gazpacho                         gazpacho   
1847919 2                        ham bone             

In [38]:
pd.reset_option('display.max_rows')

In [39]:
results_df.loc[939793,6]['name']

'domestic pig'

In [40]:
assert results_df.loc[939793,6]['name'] == 'domestic pig' # bacon

### Remaining NA's

Let's check what the remaining missing ingredients are.

In [41]:
results_df[results_df['food_id'].isnull()]

name.name         name.description  \
recipe  ingredient                                                
1703    3                    mincemeat                mincemeat   
661227  4                   beau monde               beau monde   
745912  0                      doritos              pkg doritos   
279384  7           prepared mincemeat       prepared mincemeat   
1450961 6             fajita seasoning         fajita seasoning   
1169604 0                    gremolata                gremolata   
1257944 0                    none none                none none   
        9                    none none                none none   
1883505 4                      nagaimo                  nagaimo   
1600752 7                 chunky salsa             chunky salsa   
        10                queso fresco             queso fresco   
1613023 8                       golpar                   golpar   
1609337 0                     pancetta                 pancetta   
1161383 2                   limoncello               limoncello   
1378004 10                       farro                    farro   
1307725 0                        fresh                    fresh   
1408069 3                      thymian            fresh thymian   
1984239 1                       masago                   masago   
        6                       masago                   masago   
        9                        mirin                    mirin   
480397  0                     bisquick                 bisquick   
840045  2                          &'s                      &'s   
504033  2                       season                   season   
1462944 13                queso fresco             queso fresco   
968825  1                        salsa  mild chunky-style salsa   
1725386 9                        salsa                    salsa   
1022560 9                    toothpick                toothpick   
1777734 1                          pkg                      pkg   
1097688 0                       batter                   batter   
1748614 2               liquid glucose           liquid glucose   
1936429 3                        fresh                    fresh   
1401651 4                 ra el hanout             ra el hanout   

                                          comment                 preparation  \
recipe  ingredient                                                              
1703    3                                    <NA>                        <NA>   
661227  4                         (Spice Islands)                        <NA>   
745912  0                                    <NA>                        <NA>   
279384  7                                    <NA>                        <NA>   
1450961 6                     (such as McCormick)                        <NA>   
1169604 0                                    <NA>                        <NA>   
1257944 0                         For the Muffins                        <NA>   
        9           For the Cream Cheese Frosting                        <NA>   
1883505 4                                (grated)                        <NA>   
1600752 7                                    <NA>                     drained   
        10                                   <NA>                    crumbled   
1613023 8            (angelica powder) - optional                        <NA>   
1609337 0                                    <NA>                       diced   
1161383 2                    (lemoncello liqueur)                        <NA>   
1378004 10                                   <NA>                        <NA>   
1307725 0                                      of                        <NA>   
1408069 3                                    <NA>                        <NA>   
1984239 1                           (capelin roe)                        <NA>   
        6                           (capelin roe)                        <NA>   
        9                                    <NA>

These are all highly obscure which we are fine with not showing up. Many of these are actually an issue with the ingredient parsing instead.

The can be dropped #TODO: heuristic to drop whole recipe if weight is over a threshold.

# Saving

In [42]:
results_df['food_id'].to_frame('food_id').to_feather('../../data/local/molecule/partial/food_ids/1_na_filled.feather')

In [43]:
from nbdev import nbdev_export; nbdev_export()

# Misc Investigating

In [44]:
ingredient = expanded_ingredients_df.loc[982726,	9]
ingredient = ingredient[ingredient.notnull()]
ingredient

name.name.nouns.0    raisin
Name: (982726, 9), dtype: object

In [45]:
matched_names = food_df['name'].loc[find_ingredient_food_df_matches(ingredient, food_df['name'])]
matched_names

id
1087    black raisin
Name: name, dtype: string

In [46]:
select_from_matches(ingredient, matched_names, True)

,name,whole_match_count,match_count,match_position,default_word_count,word_count
id,,,,,,
1087,black raisin,1,1,1,0,2


In [47]:
match_ingredient(ingredient, food_df['name'])

1087